In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 1. LLM 및 메모리 설정
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# 2. Few-shot 예제 설정
examples = [
    {"title": "Top Gun", "emojis": "🛩️👨‍✈️🔥"},
    {"title": "The Godfather", "emojis": "👨‍👨‍👦🔫🍝"},
    {"title": "Finding Nemo", "emojis": "🐟🔍🐢"}
]

example_template = PromptTemplate(
    input_variables=["title", "emojis"],
    template="Movie: {title}\nEmojis: {emojis}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="You are a helpful assistant that responds to movie titles with three appropriate emojis.",
    suffix="Movie: {title}\nEmojis:",
    input_variables=["title"],
    example_separator="\n"
)

# 3. Prompt 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI that provides three emojis representing a movie based on the title."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{title}"),
    ]
)

# 메모리 변수들을 획득하는 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# 4. LCEL Chain 구성
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

# 체인을 호출하는 함수 / 사용자로부터 질문을 받을 함수
def invoke_chain(title):
    result = chain.invoke({"title": title})
    memory.save_context({"input": title}, {"output": result.content})
    print(f"{title} -> {result.content}")


In [2]:
invoke_chain("Titanic")

Titanic -> 🚢💔🌊


In [3]:
invoke_chain("The Matrix")

The Matrix -> 🕶️💻🔵


In [4]:
invoke_chain("What was the first movie I asked about?")

What was the first movie I asked about? -> 🚢💔🌊
